# AutogradTensor Development

We have an issue with backprop in PyTorch. When you call `.backward()` on a torch.Tensor it immediately drops down into C++ code. However, we need to keep track of all operations in Python for use in calculating sensitivity as well as other things. We need to override `.backward()` such that we go through the backpropagation chain in Python instead of the C++ layer.

The idea here is we create a `BackpropTensor` that overrides `.backward()`. Maybe a different name, like `GradientTensor` since it should only be used on tensors where we need gradients. We can discuss naming later I suppose. Anyway, this new tensor should override `.backward()`...

In [1]:
# Run this cell to see if things work
import syft as sy
from syft.frameworks.torch.tensors.interpreters import AutogradTensor
import torch
hook = sy.TorchHook(torch)

torch.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

In [2]:
mat = sy.VirtualWorker(hook, id="matalicious")

In [3]:
a = torch.tensor([3, 2., 0], requires_grad=True)
b = torch.tensor([1, 2., 3], requires_grad=True)

a = a.send(mat, local_autograd=True)
b = b.send(mat, local_autograd=True)

In [4]:
a

(Wrapper)>AutogradTensor>[PointerTensor | me:80303136580 -> matalicious:97796238622]

In [5]:
c = a + b

In [6]:
mat._objects

{97796238622: tensor([3., 2., 0.], requires_grad=True),
 87765050530: tensor([1., 2., 3.], requires_grad=True),
 94634326384: tensor([4., 4., 3.], grad_fn=<AddBackward0>)}

In [8]:
#%pdb

In [7]:
c.backward(torch.ones(c.shape).send(mat))

In [10]:
a.grad

(Wrapper)>[PointerTensor | me:2433800062 -> matalicious:41020666215]

In [11]:
b.grad

(Wrapper)>[PointerTensor | me:4241732911 -> matalicious:65798617469]

In [12]:
mat._objects

{21159054635: tensor([3., 2., 0.], requires_grad=True),
 98458418937: tensor([1., 2., 3.], requires_grad=True),
 43639808524: tensor([3., 4., 0.], grad_fn=<MulBackward0>),
 41020666215: tensor([1., 2., 3.], grad_fn=<AddBackward0>),
 65798617469: tensor([3., 2., 0.], grad_fn=<AddBackward0>)}

(Wrapper)>AutogradTensor>[PointerTensor | me:45037380117 -> matalicious:57113225538]

In [10]:
a

(Wrapper)>[PointerTensor | me:32030935016 -> matalicious:37690125639]

In [1]:
# Run this cell to see if things work
import syft as sy
from syft.frameworks.torch.tensors.interpreters import AutogradTensor
import torch
hook = sy.TorchHook(torch)

torch.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

In [2]:
d = AutogradTensor().on(torch.tensor([3, 2., 0]))
e = AutogradTensor().on(torch.tensor([1, 2., 3]))
f = d + e

In [3]:
torch.ones_like(d)

(Wrapper)>AutogradTensor>tensor([1., 1., 1.])

In [4]:
g = d * e

In [5]:
g.grad_fn.other

tensor([1., 2., 3.])

In [6]:
g.grad_fn.other

tensor([1., 2., 3.])

In [7]:
g.backward(torch.ones(g.shape))

PureTorchTensorFoundError: 

In [11]:
g.backward(torch.ones_like(g))

PureTorchTensorFoundError: 

In [4]:
c.grad_fn(torch.ones(3))

(tensor([1., 1., 1.]), tensor([1., 1., 1.]))

In [13]:
a

(Wrapper)>AutogradTensor>tensor([3., 2., 0.])

In [6]:
# need to fix this, should show the gradient
a.grad

In [7]:
a.child.grad

AutogradTensor>tensor([1., 1., 1.])

In [8]:
b.child.grad

tensor([1., 1., 1.])

In [9]:
d = AutogradTensor().on(torch.rand(3))
e = AutogradTensor().on(torch.rand(3))

In [10]:
f = d.sin() * e.sin()

In [11]:
f.grad_fn

MulBackward

In [12]:
f.grad_fn.next_functions

(SinBackward, SinBackward)

In [13]:
f.backward()

In [15]:
e.child.grad

tensor([0.4016, 0.0379, 0.1268])

In [10]:
alice = sy.VirtualWorker(hook, id="alice")
a = torch.tensor([3, 2., 0], requires_grad=True)
b = torch.tensor([1, 2., 3], requires_grad=True)

a = a.send(alice, local_autograd=True)
b = b.send(alice, local_autograd=True)

a_torch = torch.tensor([3, 2.0, 0], requires_grad=True)
b_torch = torch.tensor([1, 2.0, 3], requires_grad=True)

c = a * b
c_torch = a_torch * b_torch

c.backward(torch.ones(c.shape).send(alice))
c_torch.backward(torch.ones(c_torch.shape))

assert (a.grad.get() == a_torch.grad).all()
assert (b.grad.get() == b_torch.grad).all()